In [5]:
import pandas as pd

In [6]:
df_tgld = pd.read_csv("/kaggle/input/tinkoff-pulse-parsing/TGLD_comments_final.csv")
df_sngs = pd.read_csv("/kaggle/input/tinkoff-pulse-parsing/SNGS_comments_final.csv")
df_lkoh = pd.read_csv("/kaggle/input/tinkoff-pulse-parsing/LKOH_comments_final.csv")
df_gazp = pd.read_csv("/kaggle/input/tinkoff-pulse-parsing/GAZP_comments_final.csv")
df_chmf = pd.read_csv("/kaggle/input/tinkoff-pulse-parsing/CHMF_comments_final.csv")
df_sber = pd.read_csv("/kaggle/input/tinkoff-pulse-parsing/SBER_comments_final.csv")
df_pikk = pd.read_csv("/kaggle/input/tinkoff-pulse-parsing/PIKK_comments_final.csv")

In [7]:
df = pd.concat([df_tgld, df_sngs, df_lkoh, df_gazp, df_chmf, df_sber, df_pikk], ignore_index = True).drop(["Unnamed: 0.1", "Unnamed: 0"], axis = 1)

In [8]:
pip install --upgrade transformers datasets


Note: you may need to restart the kernel to use updated packages.


In [9]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

MODEL_NAME = "hse-teddy-bear/xlm-roberta-russian-stock-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)


tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

2025-05-19 17:04:15.259694: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747674255.476881      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747674255.537431      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [10]:
label2id = {'buy': 2, 'sell': 0, 'neutral': 1}
id2label = {v: k for k, v in label2id.items()}
df = df[df['label'].isin(label2id.keys())]
df['label_id'] = df['label'].map(label2id)


In [11]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label_id'])


In [12]:
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.texts = df['processed text'].tolist()
        self.labels = df['label_id'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        item = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {k: v.squeeze(0) for k, v in item.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

import torch
train_dataset = SentimentDataset(train_df, tokenizer)
val_dataset = SentimentDataset(val_df, tokenizer)


In [13]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [22]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy", 
    eval_strategy = "steps",
    report_to = "none"
)

from datasets import Dataset as HFDataset
hf_train_dataset = HFDataset.from_pandas(train_df[['processed text', 'label_id']].rename(
    columns={'processed text': 'text', 'label_id': 'label'}))
hf_val_dataset = HFDataset.from_pandas(val_df[['processed text', 'label_id']].rename(
    columns={'processed text': 'text', 'label_id': 'label'}))

def preprocess_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=128,
        save_total_limit=1
    )

tokenized_train = hf_train_dataset.map(preprocess_function, batched=True)
tokenized_val = hf_val_dataset.map(preprocess_function, batched=True)

# Оставим только нужные поля
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {"accuracy": accuracy_score(labels, preds)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)

trainer.train()


Map:   0%|          | 0/49543 [00:00<?, ? examples/s]

Map:   0%|          | 0/12386 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,0.384100,0.311354,0.873971
1000,0.293900,0.274430,0.888180
1500,0.260000,0.222096,0.911352
2000,0.218000,0.213338,0.918779
2500,0.206000,0.190912,0.925157
3000,0.184100,0.183432,0.930244
3500,0.156700,0.202962,0.930405


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [24]:
eval_results = trainer.evaluate()
print(f"Accuracy on validation set: {eval_results['eval_accuracy']:.4f}")


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Accuracy on validation set: 0.9304
